In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import collections

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [3]:
def load_model(model_path):
    model = tf.saved_model.load(model_path)
    model = model.signatures['serving_default']

    return model

# Load face detector
PATH_TO_LABELS = './models/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
detection_model_path = './models/face_tf_trt_FP16'


# Load hand sign recognizer
hand_sign_classes = ["0_front", "1_back", "1_front", "2_back", "2_front", "5_front", "ILU"]
# classification_model_path = './models/hand_sign_tf_trt_FP16'
# classification_model_path = './models/hand_sign_saved_model'
classification_model_path = './models/hand_sign.h5'

face_detector = load_model(detection_model_path)
# hand_sign_recongizer = load_model(classification_model_path)
hand_sign_recongizer = tf.keras.models.load_model(classification_model_path)

In [4]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                    for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    return output_dict

In [5]:
def convert_to_absolute(im_height, im_width, box):
    box_abs = []
    box_abs = [box[0] * im_height,
               box[1] * im_width,
               box[2] * im_height,
               box[3] * im_width]
    
    return box_abs

In [6]:
def convert_to_hand_id_dict(face_id_dict):
    hand_id_dict = {}
    for face_id in face_id_dict.items():
        face_box = face_id[0]
        
        y_min = face_box[0]
        x_min = face_box[1]
        y_max = face_box[2]
        x_max = face_box[3]

        hand_box = [y_min, x_min-(x_max-x_min), y_max, x_min]

        x_offset = (hand_box[3]-hand_box[1])*0.5
        y_offset = (hand_box[2]-hand_box[0])*0.5

        hand_box[0] -= 0.5*y_offset
        hand_box[1] -= 2.5*x_offset
        hand_box[2] += 1.5*y_offset

        for i in range(len(hand_box)):
            if(hand_box[i] <= 0.0):
                hand_box[i] = 0.0
            elif(hand_box[i] >= 1.0):
                hand_box[i] = 1.0
        
        hand_id_dict[tuple(hand_box)] = face_id[1]

    return hand_id_dict

In [7]:
# Tensorflow Saved Model style
# def predict_hand_sign(image):
#     image = cv2.resize(image, (150, 150))
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
#     input_img = np.expand_dims(image, axis=0)    
#     input_img = input_img.astype(np.float32) / 255.
#     input_tensor = tf.constant(input_img)
    
#     result = hand_sign_recongizer(input_tensor)
#     preds = result['dense_1'].numpy()
#     return preds

# Keras model(*.h5) style
def predict_hand_sign(image):
    image = cv2.resize(image, (150, 150))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    input_img = np.expand_dims(image, axis=0)    
    input_img = input_img.astype(np.float32) / 255.
    
    preds = hand_sign_recongizer(input_img)
    return preds

    
def predict_hand_sign_and_visualize(image, hand_id_dict):
    im_height, im_width, _ = image.shape
    
    box_to_display_str_map = collections.defaultdict(list)
    box_to_track_ids_map = {}

    for hand_id in hand_id_dict.items():
        hand_box = convert_to_absolute(im_height, im_width, list(hand_id[0]))
        hand_img = image[int(hand_box[0]):int(hand_box[2]),
                         int(hand_box[1]):int(hand_box[3])]
        
        if hand_img.size == 0:
            continue
            
        preds = predict_hand_sign(np.array(hand_img))
        
        if np.amax(preds[0]) > 0.85:
            class_idx = np.argmax(preds[0])
            class_name = hand_sign_classes[class_idx]
            class_score = int(np.amax(preds[0])*100)
            display_str = str(class_name)
            display_str = '{}: {}%'.format(display_str, class_score)
        else:
            display_str = 'Try Again'
        
        box_to_display_str_map[hand_id[0]].append(display_str)
            
    for box, display_str in box_to_display_str_map.items():
        ymin, xmin, ymax, xmax = box
        vis_util.draw_bounding_box_on_image_array(image,
                                                  ymin,
                                                  xmin,
                                                  ymax,
                                                  xmax,
                                                  color='green',
                                                  thickness=4,
                                                  display_str_list=display_str,
                                                  use_normalized_coordinates=True)
    
#     return box_to_track_ids_map

In [15]:
# Frame's Width, Height
FRAME_WIDTH = 640
FRAME_HEIGHT = 480

# Initialize webcam feed
# video = cv2.VideoCapture(0)
video = cv2.VideoCapture('/home/young/Desktop/test/0.mp4')


ret = video.set(3, FRAME_WIDTH)
ret = video.set(4, FRAME_HEIGHT)

person_ids = list(range(10))

while(True):
    ret, frame = video.read()
    output_dict = run_inference_for_single_image(face_detector, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    face_id_dict = vis_util.visualize_boxes_and_labels_on_image_array(image=frame,
                                                                      boxes=output_dict['detection_boxes'],
                                                                      classes=output_dict['detection_classes'],
                                                                      scores=output_dict['detection_scores'],
                                                                      category_index=category_index,
                                                                      track_ids = person_ids,
                                                                      use_normalized_coordinates=True,
                                                                      line_thickness=8)
    
    hand_id_dict = convert_to_hand_id_dict(face_id_dict)
    predict_hand_sign_and_visualize(frame, hand_id_dict)
    
    cv2.imshow('Object detector', frame)
    
    # Press 'esc' to quit
    if cv2.waitKey(1) == 27:
        break

# Clean up
video.release()
cv2.destroyAllWindows()